Siguiendo https://www.youtube.com/watch?v=kCc8FmEb1nY&t=1535s. 

Status: 9:25, tokenization. 

In [1]:
# Comenzamos descargando el dataset de tiny Shakespeare para una primera implementación.
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2026-01-14 10:20:23--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2026-01-14 10:20:23 (16.9 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
# Leer el archivo
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(len(text))

1115394


In [4]:
# Así se ve el dataset
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# En forma cruda
text[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [6]:
# Cuántos caracteres?
print(len(text))

1115394


In [7]:
# Definimos el vocabulario
# Ordernados de acuerdo al código Unicode
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(chars)
print("".join(chars))

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [8]:
# Ahora nos preocupamos de la tokenización
# Transformar los caracteres de texto, strings, a una serie de números enteros

# Por simplicidad, usaremos un character-level tokenizer.
# Nota: el tamaño del diccionario y el tamaño de la secuecia de tokens son inversamente proporcionales:
# Puedes tener un diccionarioo vocabulario es pequeño (e.g. ~65 tokens para representar letras, números, y símbolos) y una secuencia de tokens más larga.
# Como ejemplo, el tokenizador te GPT2 tiene ~50k tokens. 


In [9]:
# Creemos un mapeo desde los caracteres a los enteros
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}


In [10]:
# Encoder y decode de strings

def encode(s):
    return [stoi[ch] for ch in s]

def decode(l):
    return ''.join([itos[i] for i in l])

# Ejemplos:

print(encode("hola"))
print(decode([5, 64, 2, 8]))
print(decode(encode("hola")))

[46, 53, 50, 39]
'z!.
hola


In [11]:
# Aplicamos el encoder a todo el texto
import torch

data = torch.tensor(encode(text), dtype=torch.long) # .long sets the int precision to 64 bits, standard in PyTorch
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [12]:
# Hacemos ahora el tr/val split
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [13]:
"""
Ahora, nos disponemos a entrenar el modelo. Nunca entrenamos en todo el texto porque sería
computacionalmente inviable.

Lo que hacemos en realidad es entrenar en pequeños pedazos de texto del training dataset. Estos pedazos tienen un largo máximo,
que se llama block_size.
"""

block_size = 8
print(data[:block_size+1])  


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])


In [14]:
"""
En una secuencia de 9 tokens, hay muchos ejemplos de cómo se organizan los tokens.

En una secuencia de 18, 47 probablemente sigue. En una secuencia de 18, 47, 56 probablemente sigue. Y así. Hay varios contextos.

En general, en una secuencia de N tokens, hay N-1 ejemplos que el transformer puede aprender.
"""

# Definimos el contexto
x = train_data[:block_size]
y = train_data[1:block_size+1]

print(x)
print(y)

tensor([18, 47, 56, 57, 58,  1, 15, 47])
tensor([47, 56, 57, 58,  1, 15, 47, 58])


In [15]:
# Iteramos sobre tokens
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"Cuando el contexto es {context.tolist()} el target es {target.item()}")

# Es útil entrenar sobre contexts de distinta longitud para que el transformer aprenda
# a hacer predicciones incluso con el contexto más corto, context = 1.

Cuando el contexto es [18] el target es 47
Cuando el contexto es [18, 47] el target es 56
Cuando el contexto es [18, 47, 56] el target es 57
Cuando el contexto es [18, 47, 56, 57] el target es 58
Cuando el contexto es [18, 47, 56, 57, 58] el target es 1
Cuando el contexto es [18, 47, 56, 57, 58, 1] el target es 15
Cuando el contexto es [18, 47, 56, 57, 58, 1, 15] el target es 47
Cuando el contexto es [18, 47, 56, 57, 58, 1, 15, 47] el target es 58


In [16]:
# Ahora, lo podemos generalizar
torch.manual_seed(1337)
batch_size = 4 # cuantas secuencias procesamos en paralelo encada forward/backward pass
block_size = 8 # context size

def get_batch(split):
    """
    Generate un pedazo de data con inputs x y output y.
    """

    data = train_data if split == "train" else val_data
    # Genera las posiciones random para extraer el pedazo de texto de ahí.
    # El limite superior de los indices puede ser len(data) - block_size (si no, se saldría del rango)
    # El limite inferior es 0 for default.
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [17]:
xb, yb = get_batch("train")

print("inputs")
print(xb.shape)
print(xb)
print("outputs")
print(yb.shape)
print(yb)

print("----")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"cuando el contexto es {context.tolist()} el target es {target.item()}")

# En este caso, tenemos un set de 32 tokens (independientes, hasta donde el transformer sabe)
# que alimentan al modelo en paralelo.

inputs
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
outputs
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
cuando el contexto es [24] el target es 43
cuando el contexto es [24, 43] el target es 58
cuando el contexto es [24, 43, 58] el target es 5
cuando el contexto es [24, 43, 58, 5] el target es 57
cuando el contexto es [24, 43, 58, 5, 57] el target es 1
cuando el contexto es [24, 43, 58, 5, 57, 1] el target es 46
cuando el contexto es [24, 43, 58, 5, 57, 1, 46] el target es 43
cuando el contexto es [24, 43, 58, 5, 57, 1, 46, 43] el target es 39
cuando el contexto es [44] el target es 53
cuando el contexto es [44, 53] el target es 56
cuando el contexto es [44, 53, 56] el target es 1
cuando el contexto

Implementaremos ahora uno de los modelos de lenguaje más básicos que hay: el bigram language model, que estima la probabilidad del próximo token en una secuencia considerando solamente el que le precede inmediatamente. Cada palabra depende sólo en la palabra anterior, por lo tanto es "bi-gram".

In [18]:
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        """
        En general, se tiene nn.Embedding(V, D), donde V es el número de tokens en el vocabulario y D es la dimensión del embedding space.
        """
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    # Recuerda que el nombre forward es especial en PyTorch. Si haces model(x) es como hacer model.forward(x), pero muy optimizado.
    def forward(self, idx, targets=None):
        # Tanto idx como targets son tensores de dimensions (B, T) de numeros enteros. 
        # Recuerda que los logits son los scores para cada token en el vocabulario.
        logits = self.token_embedding_table(idx) # (B, T, C)

        if targets is None:
            loss = None

        else: 
            # La documentacion de la functional cross entropy en pytorch require que el array
            # se entregue en otras dimensiones. Es un poco raro pero así es. Por lo tanto, usamos
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    # Ahora, debemos agregar un método que sea capaz de generar tokens a partir de un contexto dado.
    # En la práctica, generaremos idx de dimensiones B por T, T+1, T+2, ...
    def generate(self, idx, max_new_tokens):
        # idx es el (B, T) array de indices en el contexto actual
        for _ in range(max_new_tokens):
            # Hacer las predicciones
            logits, loss = self.forward(idx)
            # Sólo en el último step de tiempo
            logits = logits[:, -1, :] # se transforme en (B, C)
            # Aplicamos softmax para obtener las probabilidades
            probs = F.softmax(logits, dim=-1) # (B, C)
            # Sampleamos ahora de la distribución, sólo el siguiente token
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # Concatenamos el nuevo token al contexto
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [19]:
xb

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

In [20]:
yb

tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])

In [21]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)

print(logits.shape)
print(logits)

print(loss)

torch.Size([32, 65])
tensor([[ 1.6347, -0.0518,  0.4996,  ...,  0.2432,  1.1519,  0.9950],
        [ 0.3418, -0.9276,  1.2381,  ...,  1.5018, -0.5266,  0.2354],
        [ 0.1479, -0.4333,  0.5203,  ...,  0.3302,  1.5454,  1.3778],
        ...,
        [-0.5693, -0.0735,  0.7743,  ..., -0.0815, -1.1445, -0.0623],
        [ 0.4658, -0.2573, -1.0673,  ...,  1.2439,  1.3471,  1.6910],
        [-0.4553,  0.0139,  0.9309,  ...,  0.0290, -0.7568,  0.8701]],
       grad_fn=<ViewBackward0>)
tensor(5.0364, grad_fn=<NllLossBackward0>)


In [22]:
# Creamos un pequeño tensor que tiene un zero. De ahí partimos la generación porque
# el token que corresponde al token 0 es el newline character \.
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


l-QYjt'CL?jLDuQcLzy'RIo;'KdhpV
vLixa,nswYZwLEPS'ptIZqOZJ$CA$zy-QTkeMk x.gQSFCLg!iW3fO!3DGXAqTsq3pdgq


In [23]:
# Ahora comienza el entrenamiento
# Inicializamos un optimizer object
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [24]:
batch_size = 32
for step in range(10000):
    # Sampleamos un batch de datos
    xb, yb = get_batch("train")

    # Evaluamos la loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss)

tensor(2.5589, grad_fn=<NllLossBackward0>)


In [25]:
# Ya tenemos una loss un poco mejor. Veamos qué tokens genera el bigram model por ahora
print(decode(m.generate(idx, max_new_tokens=300)[0].tolist()))


Ong h hasbe pave pirance
RDe hicomyonthar's
PES:
AKEd ith henourzincenonthioneir thondy, y heltieiengerofo'dsssit ey
KINld pe wither vouprroutherccnohathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so itJas
Waketancotha:
h hay.JUCLUKn prids, r loncave w hollular s O:
HIs; ht anjx


In [26]:
# Nota que ya ha mejorado la performance. Entiende que después de cada \n newline se comienza con una mayúscula,
# El largo de las palabras ya esta mas regularizado también.

Un truco matemático para self attention. 

Para darle contexto a la next token prediction, crearemos una bag of words y usaremos su promedio para inferencia. El promedio, sin embargo, puede ser lento de calcula si no se vectoriza. Veamos cómo hacerlo

In [27]:
# Ejemplo de juguete
torch.manual_seed(1337)

B, T, C = 4, 8, 2 # batch, time, channels
x = torch.randn(B, T, C)
print(x.shape)

torch.Size([4, 8, 2])


In [28]:
# Queremos x[b, t] = promedio(i<t) * x[b, i]
# Bag of words
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # t, C
        xbow[b, t] = torch.mean(xprev, dim=0)

In [29]:
# Usaremos las matrix multiplication de Pytorch, muy optimizadas.
# Podemos recuperar el pasado de un token usando traingular matrices. Defininos weights
wei = torch.tril(torch.ones((T, T)))
print(wei)

# Normalizamos
wei = wei / wei.sum(dim=1, keepdim=True)
xbow2 = wei @ x
print(xbow2)
# Arriba, la multiplciatión arriba es (T, T) @ (B, T, C).
# Por el broadcasting, Pytorch agregará un índice al principio para que el cálculo se repita por batches.
# Queda entonces (B, T, T) @ (B, T, C) ----> (B, T, C)

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])
tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0

In [33]:
# Generaremos ahora otra forma de implementar la self attention usando Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
# Poner el future a minus infinity indica que el current token sencillamente del contexto pasado
# Despues de usat softmax en -inf sencillmanet obtendrás 0 affinity con el futuro.
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow2, xbow3)

True